In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.io import arff
import os
import scipy.stats as stats
from scipy.stats import lognorm
from scipy.stats import norm
from joblib import Parallel, delayed
from tqdm import tqdm

SEED = 14208

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset_filepath = 'datasets/wind_dataset_transformed.csv'  # Aquí se incluirían las rutas a los datasets
pred_col_name = 'WIND'  # Columna a predecir
alpha = 0.1  # Nivel de significancia

In [4]:
def process_dataset(dataset_filepath: str, pred_col_name: str):
    # Obtener la extension del archivo
    _, file_extension = os.path.splitext(dataset_filepath)

    # Cargar el dataset según la extensión
    if file_extension == '.arff':
        data = arff.loadarff(dataset_filepath)
        df = pd.DataFrame(data[0])

    elif file_extension == '.csv':
        df = pd.read_csv(dataset_filepath)

    else:
        raise ValueError("Formato no soportado, `dataset_filepath` debe tener una de las siguientes extensiones: .csv, .arff")

    # Separar el dataset en Train y Validation
    train_df, validation_df = train_test_split(df, test_size=0.2, random_state=SEED)
    y_train, y_valid = train_df[pred_col_name], validation_df[pred_col_name]
    X_train, X_valid = train_df.drop(pred_col_name, axis=1), validation_df.drop(pred_col_name, axis=1)

    # Aplicar get_dummies para variables categóricas
    X_train = pd.get_dummies(X_train)
    X_valid = pd.get_dummies(X_valid)
    X_train, X_valid = X_train.align(X_valid, join='left', axis=1, fill_value=0) 

    # Crear y entrenar el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=SEED)
    rf_model.fit(X_train, y_train)

    # Obtener las predicciones de cada árbol en el bosque
    tree_predictions = []
    for tree in rf_model.estimators_:
        tree_pred = tree.predict(X_valid)
        tree_predictions.append(tree_pred)

    tree_predictions = np.array(tree_predictions).T

    # Evaluar el modelo
    predictions = rf_model.predict(X_valid)
    mse = mean_squared_error(y_valid, predictions)

    return df, mse, predictions, tree_predictions

In [5]:
def lognormal_KS_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.0722):
    ks_results = []
    tree_predictions += shift
    for predictions in tree_predictions:
        # Estimate parameters of the lognormal distribution
        shape, loc, scale = stats.lognorm.fit(predictions, floc=0)
        
        # Perform the Kolmogorov-Smirnov test
        ks_stat, p_value = stats.kstest(predictions, 'lognorm', args=(shape, loc, scale))
        ks_results.append((ks_stat, p_value, shape, loc, scale))

    # Proportion of trees that fit the lognormal distribution
    ks_stats = [result[0] for result in ks_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in ks_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    ks_fit_proportion = len(filtered_stats) / len(ks_stats)
    
    return ks_fit_proportion, ks_stats

In [6]:
def gamma_KS_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.1323):
    ks_results = []
    tree_predictions += shift
    for predictions in tree_predictions:
        # Estimate parameters of the lognormal distribution
        shape, loc, scale = stats.gamma.fit(predictions, floc=0)
        
        # Perform the Kolmogorov-Smirnov test
        ks_stat, p_value = stats.kstest(predictions, 'gamma', args=(shape, loc, scale))
        ks_results.append((ks_stat, p_value, shape, loc, scale))

    # Proportion of trees that fit the lognormal distribution
    ks_stats = [result[0] for result in ks_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in ks_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    ks_fit_proportion = len(filtered_stats) / len(ks_stats)
    
    return ks_fit_proportion

In [7]:
def transforme_normal_SW_fit_check(tree_predictions, shift=1e-6, alpha=0.05, orig_data_stat=0.9809):
    sw_results = []
    tree_predictions = np.log1p(tree_predictions)

    for predictions in tree_predictions:
        # Perform the Shapiro-Wilk test
        shapiro_stat, shapiro_p_value = stats.shapiro(predictions)
        sw_results.append((shapiro_stat, shapiro_p_value))

    # Proportion of trees that fit the normal distribution
    sw_stats = [result[0] for result in sw_results]
    eps = alpha * orig_data_stat
    filtered_stats = [stat for stat in sw_stats if orig_data_stat-eps <= stat <= orig_data_stat+eps]
    sw_fit_proportion = len(filtered_stats) / len(sw_stats)
    
    return sw_fit_proportion

In [8]:
# def plot_lognormal_fit(predictions, dataset_name, save=False, shift=1e-6, given_index=0):
#     predictions = predictions[given_index]

#     plt.figure(figsize=(8, 6))
#     sns.histplot(predictions, kde=True, stat='density', bins=30, color='green', alpha=0.6 , label='Data')

#     shape, loc, scale = stats.lognorm.fit(predictions, floc=0)

#     x = np.linspace(min(predictions), max(predictions), 100)
#     pdf_lognormal = stats.lognorm.pdf(x, shape, loc, scale)
#     plt.plot(x, pdf_lognormal, 'r-', label='Lognormal Distribution')

#     plt.title(f"Distribucion de las predicciones | Instancia {given_index} [Validation Set]")
#     plt.legend()

#     if save:
#         plt.savefig(f'graficos/predictions/{dataset_name}/distribution_lognormal/{i}.png', format='png', dpi=300, bbox_inches='tight')

In [9]:
# def plot_normal_fit(predictions, dataset_name, save=False, shift=1e-6, given_index=0):
#     predictions = predictions[given_index]
#     predictions = np.log1p(predictions)

#     plt.figure(figsize=(8, 6))
#     sns.histplot(predictions, kde=True, stat='density', bins=30, color='green', alpha=0.6 , label='Transformed Data')

#     mean, std = np.mean(predictions), np.std(predictions)

#     x = np.linspace(min(predictions), max(predictions), 100)
#     pdf_normal = stats.norm.pdf(x, mean, std)
#     plt.plot(x, pdf_normal, 'r-', label='Normal Distribution')

#     plt.title(f"Distribucion de las predicciones | Instancia {given_index} [Validation Set]")
#     plt.legend()

#     if save:
#         plt.savefig(f'graficos/predictions/{dataset_name}/distribution_normal/{i}.png', format='png', dpi=300, bbox_inches='tight')

In [10]:
dataset, mse, predictions, tree_predictions = process_dataset(dataset_filepath, pred_col_name)

ks_lognormal_fit_proportion, ks_stats = lognormal_KS_fit_check(tree_predictions, alpha=alpha)
print(f"Proportion of trees that fit the lognormal distribution: {ks_lognormal_fit_proportion:.5f}")

sw_fit_proportion = transforme_normal_SW_fit_check(tree_predictions, alpha=alpha)
print(f"Proportion of trees that fit the normal distribution (over transformed data): {sw_fit_proportion:.5f}")

Proportion of trees that fit the lognormal distribution: 0.01684
Proportion of trees that fit the normal distribution (over transformed data): 0.83599


In [11]:
# dataset_name = os.path.basename(dataset_filepath)
# file_path = 'datasets_results.csv'

# # Check if the file exists
# if os.path.exists(file_path):
#     # Open the file
#     df = pd.read_csv(file_path)
# else:
#     # Create a new dataframe with column headers
#     df = pd.DataFrame(columns=['dataset_name', 'prop_lognormal', 'prop_normal', 'alpha', 'pred_col_name'])

# # Add a new row with values
# new_row = {'dataset_name': dataset_name, 
#            'prop_lognormal': ks_lognormal_fit_proportion, 
#            'prop_normal': sw_fit_proportion, 
#            'alpha': alpha, 
#            'pred_col_name': pred_col_name}
# #df = df.append(new_row, ignore_index=True)
# df = pd.concat([df, new_row], ignore_index=True)

# # Save the dataframe to the file
# df.to_csv(file_path, index=False)

In [12]:
import os
import pandas as pd

# Extract dataset name
dataset_name = os.path.basename(dataset_filepath)
file_path = 'datasets_results.csv'

# Check if the file exists and load or create a DataFrame
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.DataFrame(columns=['dataset_name', 'prop_lognormal', 'prop_normal', 'alpha', 'pred_col_name'])

# Create a new row as a DataFrame for concatenation
new_row = pd.DataFrame([{
    'dataset_name': dataset_name,
    'prop_lognormal': ks_lognormal_fit_proportion,
    'prop_normal': sw_fit_proportion,
    'alpha': alpha,
    'pred_col_name': pred_col_name
}])

# Concatenate the new row
df = pd.concat([df, new_row], ignore_index=True)

# Save the updated DataFrame
df.to_csv(file_path, index=False)


In [13]:
# Para eliminar las columnas "columna1" y "columna2"
#df = df.drop([26, 23, 22, 20])


In [14]:
df

,dataset_name,prop_lognormal,prop_normal,alpha,pred_col_name
0,titanic_fare_test.arff,0.019084,0.381679,0.1,Fare
1,medical_costs.csv,0.000000,0.078358,0.1,medical charges
2,salary_football.csv,0.002558,0.485934,0.1,Wage
3,Height.csv,0.000000,0.417112,0.1,childHeight
4,wine_quality.arff,0.000000,0.033846,0.1,quality
5,house_8L.arff,0.043889,0.695414,0.1,price
6,kidney.arff,0.000000,0.250000,0.1,frailty
7,flight_price.csv,0.000468,0.106692,0.1,Price
8,boston_housing.arff,0.009804,0.598039,0.1,MEDV
9,liver_disorders.arff,0.000000,0.362319,0.1,drinks


In [15]:
df.sort_values(by='prop_normal', ascending=False)

,dataset_name,prop_lognormal,prop_normal,alpha,pred_col_name
21,Carbon_Emission_transformed.csv,0.085500,0.948500,0.1,CarbonEmission
20,Carbon_Emission.csv,0.058000,0.937000,0.1,CarbonEmission
23,wind_dataset_transformed.csv,0.016844,0.835993,0.1,WIND
22,wind_dataset_transformed.csv,0.016844,0.835993,0.1,WIND
12,laptops.csv,0.010000,0.780000,0.1,Price
19,wind_dataset.csv,0.003546,0.736702,0.1,WIND
5,house_8L.arff,0.043889,0.695414,0.1,price
16,laptop.csv,0.005025,0.614322,0.1,Price
8,boston_housing.arff,0.009804,0.598039,0.1,MEDV
18,abalone.csv,0.000000,0.540670,0.1,Age
